In [25]:
import pandas as pd
from datetime import timedelta

def load_parcels_from_excel(parcels_df):
    parcels = []
    for _, row in parcels_df.iterrows():
        parcel_id = int(row['Parcel Number'])
        arrival_time = pd.to_datetime(row['Arrival Time'])
        length = float(row['Length'])
        width = float(row['Width'])
        height = float(row['Height'])
        weight = float(row['Weight'])
        feasible_outfeeds = [i for i, flag in enumerate([row['Outfeed 1'], row['Outfeed 2'], row['Outfeed 3']]) if flag]
        parcels.append(Parcel(parcel_id, arrival_time, length, width, height, weight, feasible_outfeeds))
    return sorted(parcels, key=lambda p: p.arrival_time)

class Parcel:
    def __init__(self, parcel_id, arrival_time, length, width, height, weight, feasible_outfeeds):
        self.id = parcel_id
        self.arrival_time = arrival_time
        self.length = length
        self.width = width
        self.height = height
        self.weight = weight
        self.feasible_outfeeds = feasible_outfeeds
        self.sorted = False
        self.recirculated = False


class Outfeed:
    def __init__(self, max_length=3.0):
        self.max_length = max_length
        self.current_parcels = []
        self.current_length = 0.0

    def can_accept(self, parcel):
        return self.current_length + parcel.length <= self.max_length

    def add_parcel(self, parcel):
        self.current_parcels.append(parcel)
        self.current_length += parcel.length

    def remove_parcel(self):
        if self.current_parcels:
            parcel = self.current_parcels.pop(0)
            self.current_length -= parcel.length
            return parcel
        return None


class PosiSorterSystem:
    def __init__(self, layout_df):
        self.belt_speed = layout_df.loc[layout_df['Layout property'] == 'Belt Speed', 'Value'].values[0]
        self.num_outfeeds = 3  # Given in Excel sheet
        self.outfeeds = [Outfeed(max_length=3.0) for _ in range(self.num_outfeeds)]
        self.recirculation_belt = []
        self.processed_parcels = []

    def sort_parcel(self, parcel):
        for outfeed_index in parcel.feasible_outfeeds:
            outfeed = self.outfeeds[outfeed_index]
            if outfeed.can_accept(parcel):
                outfeed.add_parcel(parcel)
                parcel.sorted = True
                print(f"Parcel {parcel.id} sorted to outfeed {outfeed_index}")
                return
        parcel.recirculated = True
        self.recirculation_belt.append(parcel)
        print(f"Parcel {parcel.id} could not be sorted and was recirculated.")

    def remove_from_outfeeds(self):
        for i, outfeed in enumerate(self.outfeeds):
            removed = outfeed.remove_parcel()
            if removed:
                print(f"Parcel {removed.id} removed from outfeed {i}")

    def run_simulation(self, parcels):
        current_time = parcels[0].arrival_time
        end_time = parcels[-1].arrival_time + timedelta(seconds=5) # 5 seconds buffer to make sure all packages are sorted
        parcel_index = 0

        while current_time <= end_time:
            print(f"\n--- Time: {current_time.time()} ---")

            # Check for arrivals
            while parcel_index < len(parcels) and parcels[parcel_index].arrival_time <= current_time:
                parcel = parcels[parcel_index]
                parcel_index += 1
                self.processed_parcels.append(parcel)
                print(f"Parcel {parcel.id} arrived – feasible outfeeds: {parcel.feasible_outfeeds}")
                self.sort_parcel(parcel)

            self.remove_from_outfeeds()
            current_time += timedelta(seconds=0.1) # Time delta can be changes, but must retain accuracy

        self.print_summary()

    def print_summary(self):
        total = len(self.processed_parcels)
        sorted_count = sum(p.sorted for p in self.processed_parcels)
        recirculated_count = sum(p.recirculated for p in self.processed_parcels)
        print("\n--- Simulation Summary ---")
        print(f"Total parcels processed: {total}")
        print(f"Parcels sorted: {sorted_count}")
        print(f"Parcels recirculated: {recirculated_count}")
        print(f"Sorting success rate: {sorted_count / total:.2%}")


def main():
    # Load Excel
    xls = pd.ExcelFile("PosiSorterData1.xlsx")
    parcels_df = xls.parse('Parcels')
    layout_df = xls.parse('Layout')

    parcels = load_parcels_from_excel(parcels_df)
    system = PosiSorterSystem(layout_df)
    system.run_simulation(parcels)


if __name__ == "__main__":
    main()





--- Time: 09:00:00 ---
Parcel 16 arrived – feasible outfeeds: [2]
Parcel 16 sorted to outfeed 2
Parcel 133 arrived – feasible outfeeds: [2]
Parcel 133 sorted to outfeed 2
Parcel 149 arrived – feasible outfeeds: [2]
Parcel 149 sorted to outfeed 2
Parcel 16 removed from outfeed 2

--- Time: 09:00:00.100000 ---
Parcel 133 removed from outfeed 2

--- Time: 09:00:00.200000 ---
Parcel 149 removed from outfeed 2

--- Time: 09:00:00.300000 ---

--- Time: 09:00:00.400000 ---

--- Time: 09:00:00.500000 ---

--- Time: 09:00:00.600000 ---

--- Time: 09:00:00.700000 ---

--- Time: 09:00:00.800000 ---

--- Time: 09:00:00.900000 ---

--- Time: 09:00:01 ---

--- Time: 09:00:01.100000 ---

--- Time: 09:00:01.200000 ---

--- Time: 09:00:01.300000 ---
Parcel 1 arrived – feasible outfeeds: [0]
Parcel 1 sorted to outfeed 0
Parcel 1 removed from outfeed 0

--- Time: 09:00:01.400000 ---

--- Time: 09:00:01.500000 ---

--- Time: 09:00:01.600000 ---

--- Time: 09:00:01.700000 ---

--- Time: 09:00:01.800000 --